<a href="https://colab.research.google.com/github/R-Saad007/Mistral-7B/blob/main/Mistral_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=6b6194597c6947e598d8185bceb81606b9cf8f918e6a0bcac606cc754f42e743
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 44.4 MB/

In [2]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [3]:
#load pdf files
loader = PyPDFDirectoryLoader("/content/sample_data/Data/")
data = loader.load()

In [4]:
print(data)

[Document(page_content='Connected City \nthrough Lighting\nLeverage your existing lighting grid to bring \nbroadband connectivity to your city', metadata={'source': '/content/sample_data/Data/Signify BrightSites Brochure.pdf', 'page': 0}), Document(page_content='What is BrightSites?\nIf you are serious about extending broadband throughout your city, BrightSites is \nwhat you’ve been waiting for.\nBrightSites transforms legacy street lighting infrastructure into a platform of \nscalable broadband connectivity. It removes barriers of entry including trenching \nfiber to every location and requires only a fraction of time and costs compared \nto traditional methods of extending broadband connectivity.\nLight is everywhere\nYour streetlight spans the length \nand breadth of your city\nSpeed to market\nNo need to trench fiber\nPlug and play installationBroadband connectivity\nEveryone gets ubiquitous high-\nspeed broadband connectivity\nLow cost \nUp to 80% lower deployment \ncostScalable e

In [5]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)

In [6]:
len(text_chunks)

4

In [7]:
#get the third chunk
text_chunks[2]

Document(page_content='Wi-FiBrightSites is aimed at transforming outdoor lighting to facilitate a wide range \nof smart city use cases such as public Wi-Fi, traffic optimization, public safety, \nsmart waste management, pollution monitoring and much more. \nThe same platform enables data densification applications such as small cells, Wi-\nFi, Fixed Wireless Access, CCTV cameras, and high-bandwidth sensors. \nThe Connectivity Grid of the Future creates a backbone for the growing demand \nof Broadband and IoT initiatives that optimize city operations and improve quality \nof life for citizens.The Connectivity Grid of the Future\nTelehealth\nData OffloadingHigh-Definition CCTV \nCameras Fixed Wireless Access4G/5G Small CellsSafety & Security\nSmart Waste ManagementPollution Monitoring\nMobile Connectivity\nHigh-Bandwidth SensorsPublic Wi-Fi\nTraffic Optimization', metadata={'source': '/content/sample_data/Data/Signify BrightSites Brochure.pdf', 'page': 2})

In [ ]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

data_config.json:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

In [9]:
#Step 07: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [10]:
#connect to google drive
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

In [ ]:
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="/content/drive/MyDrive/Model/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [ ]:
query = "List down some use cases BrightSites offer its users?"

In [ ]:
qa.run(query)

In [ ]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")